In [97]:
holding_stocks = {
    '000651':['格力电器', 0.1572, 46.00],
    '300347':['泰格医药', 0.0167, 74.20],
    '300326':['凯利泰  ', 0.0243, 18.74],
    '603882':['金域医学', 0.0125, 50.00],
    '600585':['海螺水泥', 0.0070, 40.00],
    '002821':['凯莱英  ', 0.0240, 227.0],
    '300529':['健帆生物', 0.0200, 56.30],
    '600835':['上海机电', 0.0320, 17.00],
}

attention_stocks = {
#     '603719':['良品铺子', 0.1300, ],
#     '603195':['公牛集团', 0.0200, ],
#     '688180':['君实生物', 0.01, ],
#     '603087':['甘李药业', 0.0452, ],
#     '600779':['水井坊', 0.01, ],
}

def code_fix(code):
    return code + '.SH' if code[0]=='6' else code + '.SZ'
from collections import ChainMap
all_code = {code_fix(k):v for k,v in ChainMap(holding_stocks, attention_stocks).items()}
# all_code = holding_stocks
# all_code

In [96]:
import pandas as pd
from pandas import DataFrame,Series
import tushare as ts
ts.set_token('61bd0479a5591b9a4c0112b0255ca10d29aeb1bfc2ea72cec0314725')
pro = ts.pro_api('61bd0479a5591b9a4c0112b0255ca10d29aeb1bfc2ea72cec0314725')
from datetime import datetime,timedelta
today_str = (datetime.now()- pd.tseries.offsets.DateOffset(days=2)).strftime("%Y%m%d")
pro = ts.pro_api()


Price = pro.daily(ts_code=','.join(all_code.keys()), adj='qfq', start_date=today_str, fields='ts_code,trade_date,close')
Price = DataFrame(Price.pivot('trade_date','ts_code','close').sort_index().iloc[-1,:]).reset_index()
Price.columns = ['Code','Price1']


df = DataFrame(Series(all_code)).reset_index()
for i in range(1,4):
    df[i] = df[0].map(lambda k:k[i-1])
df.drop(0, axis=1, inplace=True)
df.rename(columns = {'index':'Code',1:'Name', 2:'Hold_Ratio', 3:'Price0'}, inplace=True)



df = pd.merge(df, Price, on='Code')
df['Target'] = df['Price0']*3
df['Multiply'] = (df['Price1'] / df['Price0'])
df = df.sort_values('Multiply', ascending=False)

for col in ['Price0', 'Price1', 'Target','Multiply']:
    df[col] = df[col].map(lambda k: format(k, '0.2f'))
df['Hold_Ratio'] = df['Hold_Ratio'].map(lambda k: format(k,'.2%'))

df.index = range(len(df))
df.style.set_properties(**{'text-align': 'right'})
df

,Code,Name,Hold_Ratio,Price0,Price1,Target,Multiply
0,603882.SH,金域医学,1.25%,50.00,108.00,150.00,2.16
1,300347.SZ,泰格医药,1.67%,74.20,145.85,222.60,1.97
2,000651.SZ,格力电器,15.72%,46.00,61.03,138.00,1.33
3,600585.SH,海螺水泥,0.70%,40.00,52.70,120.00,1.32
4,300529.SZ,健帆生物,2.00%,56.30,72.30,168.90,1.28
5,002821.SZ,凯莱英,2.40%,227.00,279.96,681.00,1.23
6,600835.SH,上海机电,3.20%,17.00,19.19,51.00,1.13
7,300326.SZ,凯利泰,2.43%,18.74,14.82,56.22,0.79
